In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\eweber\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)

In [8]:
news_html = browser.html
soup = bs(news_html, 'html.parser')

In [9]:
#News Title
news_title = soup.find('div', class_='content_title').text
print(news_title)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover


In [10]:
#News Paragraph
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


In [11]:
#Photo
pic_url='https://spaceimages-mars.com'
browser.visit(pic_url)

In [12]:
browser.links.find_by_partial_text("FULL IMAGE").click()

In [13]:
pic_html = browser.html
soup = bs(pic_html, 'html.parser')
pic_url = soup.find('img', class_ = 'fancybox-image')['src']

In [14]:
featured_image_url = pic_url + pic_url
featured_image_url

'image/featured/mars3.jpgimage/featured/mars3.jpg'

In [15]:
#Facts
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)

In [32]:
facts_html = browser.html
soup = bs(facts_html, 'html.parser')
#mars_facts = soup.find('table', class_ = 'tbody')

In [33]:
mars_table = pd.read_html(facts_html)
mars_table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [48]:
mars_df = mars_table[0]
mars_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [49]:
mars_df.columns = ["Categories", "Mars", "Earth"]
mars_df

,Categories,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [50]:
print(mars_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Categories</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
   

In [134]:
#Hemisphere
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)

In [135]:
hemi_html = browser.html
soup = bs(hemi_html, "html.parser")

In [136]:
hemi = soup.find_all("div", class_="item")
#hemi

In [159]:
hemi_info = []
for x in hemi:
    h_title = x.find("h3").text
    hpic_link = x.find("a", class_="itemLink product-item")["href"]
    
    h_pics = (hemi_url + hpic_link)
    browser.visit(h_pics)
    soup = bs(h_pics, 'html.parser')
    
    h_pics_url = soup.find("img", class_="wide-image")["src"]
    print(h_title)
    print(h_pics)

C:\Users\eweber\Anaconda3\envs\PythonData\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://marshemispheres.com/cerberus.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


TypeError: 'NoneType' object is not subscriptable